# Set up

In [1]:
78160 + 86471 + 134421

299052

In [2]:
import streamlit as st
import pandas as pd
import numpy as np
from datetime import date
import io
import random


import warnings

# Ignore all warnings
warnings.filterwarnings("ignore")

In [3]:
region_name = 'DLR_SoCal'

In [4]:
parent_path = 'C:\\Users\\DONGZ027\\Desktop\\Local Crafts\\craft001_Soul'

mmmYear = 2025
ticket_price = 165

media_mapping_file = pd.read_csv(parent_path + '\data\media_label.csv')
media_mapping = media_mapping_file[media_mapping_file.region == region_name].set_index('media_code').to_dict()['media_label']

media_mapping_reverse = {v: k for k, v in media_mapping.items()}

spend_prefix = "M_P_"  
inc_prefix = "TIncT_P_" 
cpt_prefix = "CPT_P_"
mcpt_prefix = "MCPT_P_" 
minc_prefix = "nMTIncT_P_"

months_abbv = ['Oct', 'Nov', 'Dec', 'Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep']
months_full = ['October', 'November', 'December', 'January', 'February', 'March', 'April', 'May', 'June', 'July', 'August', 'September'] 

In [5]:
# Time reference
file_path = f"{parent_path}\data\DT_snowflake.csv"
time_ref = pd.read_csv(file_path)

time_ref = time_ref[[
    'FIS_WK_END_DT', 'FIS_YR_NB', 'FIS_QTR_NB', 'FIS_MO_NB', 'FIS_WK_NB', 
]].drop_duplicates() 
time_ref = time_ref[time_ref['FIS_YR_NB'] >= mmmYear]
time_ref['FIS_WK_END_DT'] = pd.to_datetime(time_ref['FIS_WK_END_DT']).dt.date
time_ref = time_ref.sort_values(by=['FIS_WK_END_DT']) 

counting_months = time_ref.groupby(['FIS_YR_NB', 'FIS_MO_NB']).size().reset_index(name='weeks_count') 
counting_months['lag_weeks'] = counting_months.groupby(['FIS_YR_NB'])['weeks_count'].cumsum().shift(fill_value=0) 
counting_months.loc[counting_months.FIS_MO_NB == 1, 'lag_weeks'] = 0

time_ref = time_ref.merge(counting_months, how = 'left', on = ['FIS_YR_NB', 'FIS_MO_NB']).drop_duplicates() 

counting_months2 = time_ref[['FIS_YR_NB', 'FIS_MO_NB']].drop_duplicates()
counting_months2 = counting_months2.reset_index(drop=True)
counting_months2['lag_months'] = counting_months2.index   

time_ref = time_ref.merge(counting_months2, how = 'left', on = ['FIS_YR_NB', 'FIS_MO_NB']).drop_duplicates() 
df_time = time_ref.reset_index()

In [6]:
# Media timing file
# **********************************************************************************
file_path = f"{parent_path}\\data\\{region_name}\\input_mediaTiming.csv"
df_curve  = pd.read_csv(file_path)
media_list = df_curve.columns


filler = []
for x in df_curve.columns:
    shard = np.zeros(104)
    values = df_curve[x].values
    print(x, values)
    shard[:len(values)] = values
    filler.append(shard)
df_curve = pd.DataFrame(np.array(filler).T, columns=df_curve.columns)
df_curve = df_curve.loc[~(df_curve == 0).all(axis=1)]
df_curve = df_curve.fillna(0) 

# Media parameters file
# **********************************************************************************
file_path = f"{parent_path}\\data\\{region_name}\\input_300pct.csv"
df_params  = pd.read_csv(file_path)
names = list(df_params.columns)
names2 = [s.replace("FABING", "ING") for s in names]
names2 = [s.replace("DIS_BAN", "BAN") for s in names2]
names2 = [s.replace("DIS_AFF", "AFF") for s in names2]
names2 = [s.replace("TlncT", "TIncT") for s in names2]
df_params.columns = names2

LTV [0.02355183 0.05145478 0.08056809 0.11261824 0.12549993 0.1277897
 0.11996051 0.10293135 0.07710975 0.05442132 0.03844865 0.02714473
 0.01917262 0.013544   0.00956589 0.00679709 0.00466586 0.0027902
 0.00145686 0.00050859 0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.        ]
STR [4.96929070e-02 1.07919490e-01 1.74658072e-01 2.10299058e-01
 1.79654444e-01 1.31616576e-01 7.03326320e-02 3.76146130e-02
 2.01342460e-02 1.04356050e-02 5.00929700e-03 2.08632900e-03
 5.18403000e-04 2.83288000e-05 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.0000000

In [7]:
def processing_spend_df(file_path):
    data = pd.read_csv(file_path)

    data = data.T.iloc[1:, :]
    data.columns = media_mapping.keys()
    data.reset_index(inplace=True)
    data.rename(columns={'index': 'FIS_MO_NB'}, inplace=True)
    data['FIS_MO_NB'] = np.arange(1, 13)
    for x in data.columns[1:]:
        data[x] = data[x].astype(float).round(1)
    return data


# scenario1 = processing_spend_df(f"{parent_path}/samples/UK Demo/DLP_UK_FY23.csv")
# scenario2 = processing_spend_df(f"{parent_path}/samples/UK Demo/DLP_UK_FY23_110.csv")
# scenario3 = processing_spend_df(f"{parent_path}/samples/UK Demo/DLP_UK_FY23_4million.csv")
# scenario4 = processing_spend_df(f"{parent_path}/samples/UK Demo/DLP_UK_4million_revised.csv")

scenario1 = processing_spend_df(f"{parent_path}/samples/DLR Demo/FY26_temp1.csv")

base_year = scenario1


In [17]:
df_time[df_time.FIS_YR_NB.between(2026, 2026 + 2)]

,index,FIS_WK_END_DT,FIS_YR_NB,FIS_QTR_NB,FIS_MO_NB,FIS_WK_NB,weeks_count,lag_weeks,lag_months
52,52,2025-10-04,2026,1,1,1,4,0,12
53,53,2025-10-11,2026,1,1,2,4,0,12
54,54,2025-10-18,2026,1,1,3,4,0,12
55,55,2025-10-25,2026,1,1,4,4,0,12
56,56,2025-11-01,2026,1,2,5,5,4,13
...,...,...,...,...,...,...,...,...,...
204,204,2028-09-02,2028,4,12,48,5,47,47
205,205,2028-09-09,2028,4,12,49,5,47,47
206,206,2028-09-16,2028,4,12,50,5,47,47
207,207,2028-09-23,2028,4,12,51,5,47,47


In [16]:
df_time

,index,FIS_WK_END_DT,FIS_YR_NB,FIS_QTR_NB,FIS_MO_NB,FIS_WK_NB,weeks_count,lag_weeks,lag_months
0,0,2024-10-05,2025,1,1,1,4,0,0
1,1,2024-10-12,2025,1,1,2,4,0,0
2,2,2024-10-19,2025,1,1,3,4,0,0
3,3,2024-10-26,2025,1,1,4,4,0,0
4,4,2024-11-02,2025,1,2,5,5,4,1
...,...,...,...,...,...,...,...,...,...
569,569,2035-09-01,2035,4,12,48,5,47,131
570,570,2035-09-08,2035,4,12,49,5,47,131
571,571,2035-09-15,2035,4,12,50,5,47,131
572,572,2035-09-22,2035,4,12,51,5,47,131


# Scenario Functions

In [8]:
def focal_check(X, spend_data, month, threshold):
    M_P_X_col = spend_prefix + X
    Inc_X_col = inc_prefix + X

    spend_array = spend_data[X].values
    S_total = spend_array.sum()
    benchmark_spend = 3 * S_total / np.count_nonzero(spend_array)

    # Handle the case when total spending is zero to avoid division by zero
    if S_total == 0:
        spend_percentages = np.zeros_like(spend_array)
    else:
        # Compute the percentage of total spend per month
        spend_percentages = spend_array / S_total
    multiplier = np.sum(spend_percentages >= (threshold / 100))

    m = spend_array[month - 1]
    M = min(m, benchmark_spend) * multiplier

    idx_closest = (np.abs(df_params[M_P_X_col] - M)).idxmin()
    M300 = df_params[M_P_X_col].iloc[idx_closest]
    Y300 = df_params[Inc_X_col].iloc[idx_closest]
    CPT300 = df_params[cpt_prefix + X].iloc[idx_closest]
    MA = df_params[minc_prefix + X].iloc[idx_closest]
    delta = df_params.at[idx_closest, 'Pct_Delta']
    MC =  delta * M

    y1 = Y300 / multiplier
    y2 = m / CPT300

    print(X, "at Month", month)
    print("multiplier = ", multiplier)
    print("monthly spend = ", m)
    print("benchmark spend = ", benchmark_spend)
    print(f"Annual index = {idx_closest}, Annual spend = {M}, Annual spend on table = {M300}")
    print(f"Closest M300: {M300}, Y300: {Y300}, CPT300: {CPT300}, MA: {MA}, MC: {MC}, delta: {delta}")
    print("y1 = ", y1, "y2 = ", y2)

    return y2

In [10]:
def compute_reward_X(X, spend_data, planning_year, threshold):
    """
    Compute the rewards over a 104-week period for a given media type and calculate
    marginal increments and costs, with an updated multiplier calculation based on a threshold.

    Args:
        X (str): The media type (e.g., "X1").
        spend_data (pd.DataFrame): DataFrame containing spending data with columns:
            - 'FIS_MO_NB': Fiscal month number.
            - Media spending columns (e.g., 'X1', 'X2', 'X3').
        planning_year (int): The fiscal year of the current spending plan.
        threshold (float): A value between 0 and 100. Months with spending percentage
            greater than threshold/100 will be counted towards the multiplier.

    Returns:
        tuple: A tuple containing:
            - reward_df (pd.DataFrame): DataFrame of shape (104, 13) with the reward curves.
            - minc_X (float): Total marginal rewards for media X over 12 months.
            - mc_X (float): Total marginal costs for media X over 12 months.
    """
    # Ensure global access to necessary dataframes
    global df_curve, df_params, df_time

    # ================================================================================================================
    # Step 1: Extract the spending array and compute the multiplier
    # ================================================================================================================
    spend_array = spend_data[X].values  # Spending for media X over 12 months
    # Compute total spending S_total
    S_total = spend_array.sum()
    if S_total > 0:
        benchmark_spend = 3 * S_total / np.count_nonzero(spend_array) # Compute the benchmark, or upper bound, for monthly spending
    else:
        benchmark_spend = 1000000

    # Handle the case when total spending is zero to avoid division by zero
    if S_total == 0:
        spend_percentages = np.zeros_like(spend_array)
    else:
        # Compute the percentage of total spend per month
        spend_percentages = spend_array / S_total

    # Compute the multiplier based on the threshold
    multiplier = np.sum(spend_percentages >= (threshold / 100))

    # If multiplier is zero, set it to 1 to avoid division by zero later
    if multiplier == 0:
        multiplier = 1

    # ================================================================================================================
    # Step 2: Compute the timing stuff
    # ================================================================================================================
    # Prepare a list to hold the 104-length arrays for each month
    monthly_arrays = []

    # Get fiscal years and months from spend_data
    fiscal_months = spend_data.FIS_MO_NB.values.flatten()

    # Filter df_time for the relevant fiscal year
    df_time_filtered = df_time[df_time['FIS_YR_NB'] == planning_year]

    # Group by fiscal month and count the number of weeks in each month
    weeks_in_month = df_time_filtered.groupby('FIS_MO_NB')['FIS_WK_NB'].nunique()
    weeks_in_month = weeks_in_month.reindex(range(1, 13), fill_value=0)

    # Compute cumulative weeks to determine the starting week for each month
    cumulative_weeks = weeks_in_month.cumsum()
    start_weeks = [0] + cumulative_weeks.values.tolist()[:-1]

    timeline = df_time[df_time.FIS_YR_NB.between(planning_year, planning_year + 2)].shape[0]
    timeline = np.zeros(timeline)

    # Initialize total marginal increments and costs
    minc_X = 0.0
    mc_X = 0.0

    # ================================================================================================================
    # Loop over each month to compute the rewards
    # ================================================================================================================
    for idx, S_mo in enumerate(spend_array):
        fiscal_month = fiscal_months[idx]

        if S_mo == 0:
            # If the spending is zero, append an array of zeros
            monthly_array = timeline.copy()  # Create a an array of all 0s
            monthly_arrays.append(monthly_array)
            continue

        # Step 2: Calculate annualized spending and find the closest match in df_params
        S_yr = min(S_mo, benchmark_spend) * multiplier
        M_P_X_col = spend_prefix + X
        TIncT_P_X_col = inc_prefix + X
        nMTIncT_P_X_col = minc_prefix + X  # Marginal rewards column
        Pct_Delta_col = 'Pct_Delta'   

        # Find the index of the closest spending value in df_params
        idx_closest = (np.abs(df_params[M_P_X_col] - S_yr)).idxmin()

        if idx_closest == 0:
            reward_mo = 0
        else:
            # Logic 1: read row (annual) increment and divide by the multiplie
            # ............................................................
            reward_mo = df_params[inc_prefix + X].iloc[idx_closest] / multiplier
            
            # Logic 2: use row CPT and actual monthly spend
            # ................................................................
            # cpt_yr = df_params[cpt_prefix + X].iloc[idx_closest]
            # reward_mo = S_mo / cpt_yr

        # Calculate minc_mo and mc_mo for marginal cost per reward
        minc_mo = df_params.at[idx_closest, nMTIncT_P_X_col]
        mc_mo = df_params.at[idx_closest, Pct_Delta_col] * S_yr
        # mc_mo = 0.001 * S_yr


        # Accumulate total marginal increments and costs
        minc_X += minc_mo
        mc_X += mc_mo

        # Step 3: Generate the reward curve for the month
        timing_curve = df_curve[X].values  # Timing curve of length 52
        monthly_reward_curve = reward_mo * timing_curve
        # monthly_arrays.append(monthly_reward_curve)

        # Step 4: Create a 3yr-length array with appropriate leading zeros
        start_week = start_weeks[fiscal_month - 1]
        end_week = start_week + len(monthly_reward_curve) 

        monthly_array = timeline.copy()  
        monthly_array[start_week:end_week] = monthly_reward_curve 
        monthly_arrays.append(monthly_array)
    
    
    # # #Aggregate the monthly arrays into a final reward curve
    # # monthly_arrays = [arr[:len(timeline)] for arr in monthly_arrays]  # Ensure each array is at most 104 elements
    # # # Pad any shorter arrays to exactly 104 elements
    # # monthly_arrays = [np.pad(arr, (0, len(timeline) - len(arr)), 'constant') if len(arr) < len(timeline) else arr for arr in monthly_arrays]
    reward_X = np.sum(monthly_arrays, axis=0)


    # Construct the output DataFrame
    columns = [f'Month_{i+1}' for i in range(12)] + ['aggregated']
    data = np.column_stack(monthly_arrays + [reward_X])
    reward_df = pd.DataFrame(data, columns=columns)

    return reward_df, minc_X, mc_X




In [11]:
def exmaine_media(media, spend_plan, year, ratio):
    time = df_time[df_time.FIS_YR_NB.between(year, year + 2)][['FIS_YR_NB', 'FIS_WK_NB', 'FIS_MO_NB']]
    time["Month"] = time['FIS_YR_NB'].astype(str) + '-' + time['FIS_MO_NB'].astype(str)


    time2 =  df_time[df_time.FIS_YR_NB.between(year, year + 2)][['FIS_YR_NB', 'FIS_MO_NB']] .drop_duplicates()
    time2 = time2.reset_index(drop=True)
    time2.reset_index(inplace=True)
    time = time2.merge(time, how='left', on=['FIS_YR_NB', 'FIS_MO_NB'])

    check = compute_reward_X(media, spend_plan, year, ratio)[0]
    items = check.columns
    check['Month'] = time['Month'].values
    check['Ind'] = time['index'].values
    check.sort_values(by=['Ind'], inplace=True)

    check = check.groupby(['Ind', 'Month'])[items].sum().reset_index()
    check.iloc[:, 1:] = np.round(check.iloc[:, 1:], 1)
    return check

In [ ]:
def compute_plan_reward(spend_data, planning_year, lead_years, lag_years, threshold):
    """
    Compute the reward curves for all medias over an extended time frame, broken down by month and aggregated.

    Args:
        spend_data (pd.DataFrame): DataFrame containing spending data with columns:
            - 'FIS_MO_NB': Fiscal month number.
            - Media spending columns (e.g., 'X1', 'X2', 'X3').
        lead_years (int): Number of years to add as leading zeros.
        lag_years (int): Number of years to add as trailing zeros.2
        planning_year (int): The fiscal year of the current spending plan.
        threshold (float): Threshold value for reward calculations.

    Returns:
        list: A list of 13 numpy arrays:
            - First 12 arrays represent the monthly rewards summed across all media
            - Last array represents the aggregated rewards summed across all media
            Each array includes leading and trailing zeros.
    """
    # Ensure global access to necessary dataframes
    # global df_curve, df_params, df_time

    # Step 1: Get the list of media columns from spend_data
    medias = spend_data.columns.tolist()[1:]

    # Initialize lists to store the monthly and aggregated reward arrays
    monthly_rewards = [[] for _ in range(12)]  # One list for each month
    aggregated_rewards = []  # List for aggregated rewards

    # Loop over each media to compute its reward curves
    for media in medias:
        # Extract spending data for the media
        media_spend_data = spend_data[['FIS_MO_NB', media]]

        # Call compute_reward_X for the current media
        reward_df = compute_reward_X(media, media_spend_data, planning_year, threshold)[0]

        # Extract monthly columns and aggregated column
        for month in range(12):
            month_col = f'Month_{month+1}'
            monthly_rewards[month].append(reward_df[month_col].values)
        
        # Extract and store aggregated rewards
        aggregated_rewards.append(reward_df['aggregated'].values)

    # Step 2: Sum the reward arrays across all media for each month and aggregated
    summed_monthly_rewards = [np.sum(month_arrays, axis=0) for month_arrays in monthly_rewards]
    summed_aggregated_rewards = np.sum(aggregated_rewards, axis=0)

    # Step 3: Add leading and trailing zeros to each array
    leading_zeros = np.zeros(lead_years * 52)
    trailing_zeros = np.zeros(lag_years * 52)

    # Create final list of extended arrays (12 monthly + 1 aggregated)
    extended_reward_curves = []
    
    # Process monthly arrays
    for monthly_reward in summed_monthly_rewards:
        extended_monthly = np.concatenate([leading_zeros, monthly_reward, trailing_zeros])
        extended_reward_curves.append(extended_monthly)
    
    # Process aggregated array
    extended_aggregated = np.concatenate([leading_zeros, summed_aggregated_rewards, trailing_zeros])
    extended_reward_curves.append(extended_aggregated)

    return extended_reward_curves


In [12]:
compute_plan_reward(scenario1, 2025, 0, 0, 0.00001)

[array([4.22823778e+03, 9.22303068e+03, 1.41028207e+04, 1.71702926e+04,
        1.49761545e+04, 1.13321175e+04, 7.32697508e+03, 4.84114201e+03,
        3.19899063e+03, 2.11387270e+03, 1.41755650e+03, 9.57931658e+02,
        6.59203977e+02, 4.65972052e+02, 3.43660462e+02, 2.65067533e+02,
        2.04908159e+02, 1.54240365e+02, 1.16387801e+02, 8.81718534e+01,
        6.56926849e+01, 5.29746416e+01, 4.24961405e+01, 3.38449150e+01,
        2.88493416e+01, 2.45911232e+01, 1.69023568e+01, 1.03484685e+01,
        4.76194193e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.000000

In [13]:
def plan_forecast_craft(spend_data, planning_year, lead_years, lag_years, cutoff):
    """
    Generate monthly and quarterly forecast tables based on spending data and reward calculations.

    Args:
        spend_data (pd.DataFrame): DataFrame containing spending data with columns:
            - 'FIS_MO_NB': Fiscal month number.
            - Media spending columns (e.g., 'NTV', 'ING', 'STR').
        planning_year (int): The fiscal year of the current spending plan.
        lead_years (int): Number of years to add as leading zeros.
        lag_years (int): Number of years to add as trailing zeros.
        cutoff (float): Threshold value for reward calculations.

    Returns:
        tuple: A tuple containing:
            - craft_mo (pd.DataFrame): DataFrame with monthly forecast data.
            - craft_qtr (pd.DataFrame): DataFrame with quarterly forecast data.
    """
    df_time_scenario = df_time[df_time['FIS_YR_NB'].between(planning_year - lead_years, planning_year + lag_years)]
    weekly_table = df_time_scenario[['FIS_WK_END_DT', 'FIS_YR_NB', 'FIS_QTR_NB', 'FIS_MO_NB']]
    results = compute_plan_reward(spend_data, planning_year, lead_years, lag_years, cutoff)

    names = []
    for i in range(len(results)-1):
        serl = list(results[i])
        if len(serl) < weekly_table.shape[0]:
            serl = serl + [0] * (weekly_table.shape[0] - len(serl))
        if len(serl) > weekly_table.shape[0]:
            serl = serl[:weekly_table.shape[0]]
        col_name = str(planning_year) + ' ' + months_abbv[i]
        names.append(col_name)
        weekly_table[col_name] = serl

    # Monthly Table
    # ================================================================================================================
    monthly_table = weekly_table.groupby(['FIS_YR_NB', 'FIS_MO_NB'])[names].sum().reset_index()
    rewards = monthly_table.iloc[:, 2:].values.T
    monthly_table.FIS_MO_NB.replace(dict(zip(range(1, 13), months_abbv)), inplace=True) 
    monthly_table['timeline'] = monthly_table.FIS_YR_NB.astype(str) + " " + monthly_table.FIS_MO_NB.astype(str) 
    
    shard1 = pd.DataFrame({'Spending Month': names, "Spend": spend_data.iloc[:, 1:].sum(axis = 1).values})
    shard2 = pd.DataFrame(rewards)
    shard2.columns = monthly_table['timeline'].values
    craft_mo = pd.concat([shard1, shard2], axis=1) 


    # Monthly Table
    # ================================================================================================================
    quarter_table = weekly_table.groupby(['FIS_YR_NB', 'FIS_QTR_NB'])[names].sum().reset_index()
    rewards = quarter_table.iloc[:, 2:].values.T 
    rewards = rewards.reshape(4, 3, -1) # Turning monthly tracking into quarterly tracking
    rewards = rewards.sum(axis = 1)
    quarter_table.FIS_QTR_NB = quarter_table.FIS_QTR_NB.astype(str)
    quarter_table['timeline'] = quarter_table.FIS_YR_NB.astype(str) + " Q" + quarter_table.FIS_QTR_NB.astype(str)

    names = [str(planning_year) + " Q" + str(x) for x in range(1, 5)]
    shard1 = pd.DataFrame({'Spending Quarter': names, 
                           "Spend": spend_data.iloc[:, 1:].values.sum(axis = 1).reshape(4, 3).sum(axis = 1)})
    shard2 = pd.DataFrame(rewards)
    shard2.columns = quarter_table['timeline'].values
    craft_qtr = pd.concat([shard1, shard2], axis=1) 


    return craft_mo, craft_qtr
    

In [14]:
def build_plan_summary(spend_data, planning_year, threshold, unit_revenue):
    """
    Build a summary report of the spending plan with total spend, total reward,
    cost per reward, and marginal cost per reward for each media and aggregated.

    Args:
        spend_data (pd.DataFrame): DataFrame containing spending data with columns:
            - 'FIS_MO_NB': Fiscal month number.
            - Media spending columns (e.g., 'YOT', 'FAB').
        planning_year (int): The fiscal year of the current spending plan.

    Returns:
        pd.DataFrame: A DataFrame 'plan_summary' with columns:
            - 'Total Spend'
            - 'Total Reward'
            - 'Cost per Reward'
            - 'Marginal Cost per Reward'
        and rows:
            - 'Total' (aggregated over all medias)
            - One row per media (e.g., 'YOT', 'FAB')
    """
    # Ensure nonlocal access to necessary dataframes
    # nonlocal df_curve, df_params, df_time

    # Initialize dictionaries to store summary data
    total_spend_dict = {}
    total_reward_dict = {}
    minc_X_dict = {}
    mc_X_dict = {}

    # List of media columns (exclude 'FIS_MO_NB')
    medias = spend_data.columns.tolist()[1:]

    # Loop over each media to compute summary metrics
    for media in medias:
        # Extract spending data for the media
        media_spend_data = spend_data[['FIS_MO_NB', media]]


        # Total spend for media: sum of monthly spend
        total_spend = media_spend_data[media].sum()
        total_spend_dict[media] = total_spend


        # Call compute_reward_X for the current media
        reward_df, minc_X, mc_X = compute_reward_X(media, media_spend_data, planning_year, threshold)
        

        # Total reward for media: sum of the 'aggregated' column
        total_reward = reward_df['aggregated'].sum()
        total_reward_dict[media] = total_reward

        # Store minc_X and mc_X
        minc_X_dict[media] = minc_X
        mc_X_dict[media] = mc_X


    # Calculate aggregated totals
    total_spend_agg = sum(total_spend_dict.values())
    total_reward_agg = sum(total_reward_dict.values())
    minc_X_agg = sum(minc_X_dict.values())
    mc_X_agg = sum(mc_X_dict.values())


    # Build the summary DataFrame
    plan_summary = pd.DataFrame(columns=['Total Spend', 'Total Attendance', 'Cost per Attendance', 'Marginal Cost per Attendance'])

    # Add aggregated totals to the summary
    plan_summary.loc['Total'] = {
        'Total Spend': total_spend_agg,
        'Total Attendance': total_reward_agg,
        'Cost per Attendance': total_spend_agg / total_reward_agg if total_reward_agg != 0 else np.nan,
        'Marginal Cost per Attendance': mc_X_agg / minc_X_agg if minc_X_agg != 0 else np.nan
    }

    # Add individual media data to the summary
    for media in medias:
        total_spend = total_spend_dict[media]
        total_reward = total_reward_dict[media]
        minc_X = minc_X_dict[media]
        mc_X = mc_X_dict[media]

        plan_summary.loc[media] = {
            'Total Spend': total_spend,
            'Total Attendance': total_reward,
            'Cost per Attendance': total_spend / total_reward if total_reward != 0 else np.nan,
            'Marginal Cost per Attendance': mc_X / minc_X if minc_X != 0 else np.nan
        }

    # Reorder rows: aggregated totals first, then individual medias
    plan_summary = plan_summary.reset_index().rename(columns={'index': 'Media'})
    plan_summary = plan_summary[['Media', 'Total Spend', 'Total Attendance', 'Cost per Attendance', 'Marginal Cost per Attendance']]

    # Add ROAS and MROAS
    plan_summary['ROAS'] = unit_revenue * plan_summary['Total Attendance'] / plan_summary['Total Spend']
    plan_summary['MROAS'] = 1 + ((unit_revenue - plan_summary['Marginal Cost per Attendance']) / plan_summary['Marginal Cost per Attendance'])


    # Round up columns
    plan_summary['Total Spend'] = plan_summary['Total Spend'].astype(int)
    plan_summary['Total Attendance'] = plan_summary['Total Attendance'].astype(int)
    plan_summary['Cost per Attendance'] = plan_summary['Cost per Attendance'].round(1)
    plan_summary['Marginal Cost per Attendance'] = plan_summary['Marginal Cost per Attendance'].round(1)
    plan_summary['ROAS'] = plan_summary['ROAS'].round(1)
    plan_summary['MROAS'] = plan_summary['MROAS'].round(1)


    return plan_summary

In [15]:
def optimization_summary(plan0, plan1, planning_year, planning_months, rewards, cutoff, unit_revenue):

    # Table 1 - Aggregate Summary
    # ****************************************************************************************************************
    contents = ['Total Spend', 'Total Reward', 'Total Reward in Planning Period', 'Cost per Reward', 'Marginal Cost per Reward', 'ROAS', "MROAS"]
    summary0 = build_plan_summary(plan0, planning_year, cutoff, unit_revenue)
    summary1 = build_plan_summary(plan1, planning_year, cutoff, unit_revenue)
    spend0 = int(plan0[plan0.FIS_MO_NB.isin(planning_months)].iloc[:, 1:].values.sum())
    spend1 = int(plan1[plan1.FIS_MO_NB.isin(planning_months)].iloc[:, 1:].values.sum())

    total_original = summary0.iloc[0, 1:].values.tolist()
    planning_original = rewards[0]
    collect_original = total_original[:2] + [planning_original] + total_original[2:]

    total_optimized = summary1.iloc[0, 1:].values.tolist()
    planning_optimized = rewards[1]
    collect_optimized = total_optimized[:2] + [planning_optimized] + total_optimized[2:]

    table1 = pd.DataFrame({
        'Contents': contents,
        'Original': collect_original,
        'Optimized': collect_optimized
    })

    array_original = table1.Original.values
    array_optimized = table1.Optimized.values

    
    array_original[0] = spend0
    array_original[3] = np.round(array_original[0] / array_original[2], 1) 
    array_original[5] = np.round(unit_revenue * array_original[2] / array_original[0], 1)
    table1['Original'] = array_original

    array_optimized[0] = spend1
    array_optimized[3] = np.round(array_optimized[0] / array_optimized[2], 1)
    array_optimized[5] = np.round(unit_revenue * array_optimized[2] / array_optimized[0], 1)
    table1['Optimized'] = array_optimized


    # table1.loc[table1.Contents == 'Cost per Reward', 'Original'] = np.round(table1.loc[table1.Contents == 'Total Spend', 'Original'] / table1.loc[table1.Contents == 'Total Reward in Planning Period', 'Original'], 1)
    # table1.loc[table1.Contents == 'Cost per Reward', 'Optimized'] = np.round(table1.loc[table1.Contents == 'Total Spend', 'Optimized'] / table1.loc[table1.Contents == 'Total Reward in Planning Period', 'Optimized'], 1)

    # table1.loc[table1.Contents == 'ROAS', 'Original'] =  np.round((unit_revenue * table1.loc[table1.Contents == 'Total Reward in Planning Period', 'Original'].values[0]) / table1.loc[table1.Contents == 'Total Spend', 'Original'], 1)
    # table1.loc[table1.Contents == 'ROAS', 'Optimized'] =  np.round((unit_revenue * table1.loc[table1.Contents == 'Total Reward in Planning Period', 'Optimized'].values[0]) / table1.loc[table1.Contents == 'Total Spend', 'Optimized'], 1)


    table1['Change'] = table1['Optimized'] - table1['Original']
    table1['Change (%)'] = np.round((table1.Optimized / table1.Original - 1) * 100, 1)


    # Table 2 - Media Level Summary
    # ****************************************************************************************************************
    values = []
    for x in media_list:
        collect0 = np.round(summary0[summary0.Media == x].iloc[:, 1:].values, 1)[0]
        spend0 = collect0[0]
        reward0_total = collect0[1]
        reward0_plan = np.round(compute_reward_X(x, plan0, planning_year, cutoff)[0]['aggregated'].values[:52].sum(),1) 
        cpa0 = collect0[2]
        mcpa0 = collect0[3]
        values.append([x, 'original', spend0, reward0_total, reward0_plan, cpa0, mcpa0])

        collect1 = np.round(summary1[summary1.Media == x].iloc[:, 1:].values, 1)[0]
        spend1 = collect1[0]
        reward1_total = collect1[1]
        reward1_plan = np.round(compute_reward_X(x, plan1, planning_year, cutoff)[0]['aggregated'].values[:52].sum(),1)
        cpa1 = collect1[2]
        mcpa1 = collect1[3]
        values.append([x, 'optimized', spend1, reward1_total, reward1_plan, cpa1, mcpa1])

    table2 = pd.DataFrame(values, columns=['Media', 'Version', 'Total Spend', 'Total Reward', 'Total Reward in Planning Period', 'Cost per Reward', 'Marginal Cost per Reward'])
    table2.iloc[:, 2:] = np.round(table2.iloc[:, 2:], 1)
    
    craft = pd.DataFrame(table2['Media'])
    for x in table2.columns[2:]:
        df = table2[['Media', 'Version', x]]
        hierarchical_df = df.set_index(['Media', 'Version']).unstack(level=1)
        hierarchical_df.columns = [f'{col[0]} ({col[1]})' for col in hierarchical_df.columns]
        hierarchical_df = hierarchical_df.reset_index()
        hierarchical_df[x + ' Change(%)'] = 100 * (hierarchical_df[x + ' (optimized)'] / hierarchical_df[x + ' (original)'] - 1)
        hierarchical_df[x + ' Change(%)'] = np.round(hierarchical_df[x + ' Change(%)'], 1)
        craft = craft.merge(hierarchical_df, how = 'left', on = "Media").drop_duplicates()
    table2 = craft

    return [plan1, table1, table2]

# Maximizer

In [15]:
check = plan_forecast_craft(scenario3, 2024, 0, 0, 0.001)[1]
check.iloc[:, -1].sum()

264936.31581619865

In [16]:
check = plan_forecast_craft(scenario4, 2024, 0, 0, 0.001)[1]
check.iloc[:, -1].sum()

299026.5883901889

### Utility functions

In [17]:
def revise_spend(spend_plan, newSpend_array):
    spend_plan_revised = spend_plan.copy()
    spend_plan_revised.iloc[:, 1:] = spend_plan.iloc[:, 1:]
    for i, month in enumerate(planning_months):
        spend_plan_revised.loc[spend_plan_revised['FIS_MO_NB'] == month, spend_plan.columns[1:]] = newSpend_array[i*len(spend_plan.columns[1:]):(i+1)*len(spend_plan.columns[1:])]
    return spend_plan_revised

In [18]:
def current_reward_calculator(plan, planning_year, planning_months, planning_weeks, cutoff):
    plan_rewards = compute_plan_reward(plan, planning_year, 1, 0, cutoff)
    current_reward = 0 
    for x in planning_months:
        current_reward += plan_rewards[x-1][planning_weeks].sum() 
    return current_reward

### Inputs

In [20]:
medias = scenario3.columns[1:]
bounds = pd.DataFrame({
    'Media' : medias,
    'LB' : [0.8] * len(medias),
    'UB' : [1.2] * len(medias)
})

bounds

,Media,LB,UB
0,NTV,0.8,1.2
1,STR,0.8,1.2
2,BAN,0.8,1.2
3,RAD,0.8,1.2
4,AFF,0.8,1.2
5,CEM,0.8,1.2
6,SEM,0.8,1.2
7,Total_Social,0.8,1.2


In [68]:
spend_plan = scenario1.copy()
planning_months = [10, 11, 12]
impact_months = [10, 11, 12, 13, 14, 15] 
planning_year = 2024
base_year = base_year
df_bounds = bounds
cutoff = 0.001 

### 1) Initial Setup

In [69]:
# Step 0: Filter df_time and get planning_weeks
# ================================================================================================================
df_time_filtered = df_time[df_time['FIS_YR_NB'].isin([planning_year - 1, planning_year, planning_year + 1])].reset_index(drop=True)
df_time_filtered.loc[df_time_filtered.FIS_YR_NB == planning_year + 1, 'FIS_MO_NB'] = df_time_filtered.loc[df_time_filtered.FIS_YR_NB == planning_year + 1, 'FIS_MO_NB'] + 12

impact_weeks = df_time_filtered[
    (df_time_filtered['FIS_YR_NB'].isin([planning_year, planning_year + 1])) & 
    (df_time_filtered['FIS_MO_NB'].isin(impact_months))
].index.tolist()

In [70]:
# Step 1: Compute reward_credit
# ================================================================================================================
reward_credit_year0 = compute_plan_reward(base_year, planning_year - 1, 0, 1, cutoff)[-1][impact_weeks].sum()
reward_credit_year1 = 0
if min(planning_months) > 1:
    plan_rewards = compute_plan_reward(spend_plan, planning_year, 1, 0, cutoff) 
    for i in np.arange(min(planning_months) - 1):
        reward_credit_year1 += plan_rewards[i][impact_weeks].sum() 
reward_credit = reward_credit_year0 + reward_credit_year1

In [71]:
# Step 2: Set spendings in the spending month to 0
# ================================================================================================================
spend_plan_wiped = spend_plan.copy()
spend_plan_wiped.loc[spend_plan_wiped['FIS_MO_NB'].isin(planning_months), spend_plan_wiped.columns[1:]] = 0

In [77]:
month = 10
start = df_time_filtered[(df_time_filtered['FIS_YR_NB'] == planning_year) & (df_time_filtered['FIS_MO_NB'] == month)]['index'].values[0]
landing_weeks = df_time_filtered[
        (df_time_filtered['FIS_YR_NB'].isin([planning_year, planning_year+1])) & 
        (df_time_filtered['FIS_MO_NB'].isin(impact_months))
    ] 
landing_weeks = landing_weeks['index'].values - start

cumulative_indices = df_curve.iloc[landing_weeks, :].sum()

In [79]:
# Step 3: For each {media, month}, compute the potential gain percentgae (due to media timing)
# ================================================================================================================
result_data = []
for month in planning_months:
    # Compute available weeks
    start = df_time_filtered[(df_time_filtered['FIS_YR_NB'] == planning_year) & (df_time_filtered['FIS_MO_NB'] == month)]['index'].values[0]
    landing_weeks = df_time_filtered[
            (df_time_filtered['FIS_YR_NB'].isin([planning_year, planning_year+1])) & 
            (df_time_filtered['FIS_MO_NB'].isin(impact_months))
        ] 
    landing_weeks = landing_weeks['index'].values - start
    # Compute cumulative indices for each media
    cumulative_indices = df_curve.iloc[landing_weeks, :].sum()
    result_data.append(cumulative_indices.tolist())
cumu_timing_df = pd.DataFrame(result_data, index=planning_months)
cumu_timing_df.columns = media_list


In [81]:
# Step 4: Flatten media timing index, media names, and media spend
# ================================================================================================================
array_cumu_timing = cumu_timing_df.values.flatten()
array_media_entries = np.tile(media_list, len(planning_months))
spend0 = []
for month in planning_months:
    spend0.extend(spend_plan[spend_plan['FIS_MO_NB'] == month].iloc[:, 1:].values.flatten())
spend0 = np.array(spend0)

In [84]:
# Step 5: Form the lower bound array and make the array of multipliers at {month, media} level
# ================================================================================================================
array_lb = []
for spend, media in zip(spend0, array_media_entries):
    lb_pct = df_bounds.loc[df_bounds['Media'] == media, 'LB'].values[0]
    ub_pct = df_bounds.loc[df_bounds['Media'] == media, 'UB'].values[0]
    
    array_lb.append(spend * lb_pct)
array_lb = np.array(array_lb)

spend_lb = spend_plan_wiped.copy()
for i, month in enumerate(planning_months):
    spend_lb.loc[spend_lb['FIS_MO_NB'] == month, spend_plan.columns[1:]] = array_lb[i*len(spend_plan.columns[1:]):(i+1)*len(spend_plan.columns[1:])]


multipliers = [] 
for x in media_list:
    x_sum = spend_lb[x].values.sum()

    multipliers.append(len([x for x in spend_lb[x].values if x/x_sum >= (cutoff / 100)]))

multipliers = pd.DataFrame({
    'Media': media_list,
    'LB': multipliers
})
multipliers = np.tile(multipliers.LB.values, len(planning_months))

In [86]:
spend_lb

,FIS_MO_NB,NTV,STR,BAN,RAD,AFF,CEM,SEM,Total_Social
0,1,946806.50,363350.70,39416.60,0.0,12032.0,0.0,110936.40,426092.00
1,2,41165.50,0.00,19200.60,0.0,0.0,0.0,49980.40,132612.40
2,3,0.00,0.00,11728.40,0.0,0.0,0.0,63301.80,103799.20
3,4,0.00,360526.60,39690.90,0.0,12323.0,0.0,47893.40,404875.70
4,5,957168.60,0.00,15770.90,0.0,0.0,0.0,47014.00,76967.50
5,6,885603.40,432833.00,42717.30,0.0,9628.0,0.0,69509.30,467814.60
6,7,0.00,0.00,30675.60,0.0,1296.0,0.0,52844.20,146609.80
7,8,739623.50,234384.70,42306.70,0.0,11719.0,0.0,85675.90,454750.40
8,9,105660.50,0.00,46977.50,0.0,0.0,0.0,142088.40,259544.10
9,10,0.00,0.00,20161.04,0.0,278.4,0.0,77197.52,151648.32


In [290]:
# Step 6: Create the reward panel
# ================================================================================================================
reward_panels = []
for i, (media_entry, cumu_timing) in enumerate(zip(array_media_entries, array_cumu_timing)):
    multiplier = multipliers[i]

    # Define variable names
    spend_varname = spend_prefix + media_entry
    inc_varname = inc_prefix + media_entry
    cpt_varname = cpt_prefix + media_entry

    # Create reward_panel for this entry
    reward_panel = df_params[['PCT_Change', spend_varname, inc_varname]].copy()

    # Adjust spend and inc columns
    reward_panel[spend_varname] /= multiplier
    reward_panel[inc_varname] = (reward_panel[inc_varname] / multiplier) * cumu_timing

    # Create CPT column
    reward_panel[cpt_varname] = reward_panel[spend_varname] / reward_panel[inc_varname]

    # Rename columns
    reward_panel.columns = ['pct', f'S_{i}', f'R_{i}', f'CPT_{i}']
    
    reward_panels.append(reward_panel)
    
# Combine all reward panels and rename to df_params_monthly
df_params_monthly = pd.concat(reward_panels, axis=1)

# Keep only one 'pct' column
df_params_monthly = df_params_monthly.loc[:, ~df_params_monthly.columns.duplicated()]
df_params_monthly = df_params_monthly.fillna(0)

In [295]:
# Step 7 : form the upperbound array
# ================================================================================================================
array_ub = []
array_300 = []
for i, media in enumerate(array_media_entries):
    max_spend = df_params_monthly.loc[df_params_monthly['pct'] == 300, f'S_{i}'].values[0]
    array_300.append(max_spend)
array_300 = np.array(array_300)
for spend, media in zip(spend0, array_media_entries):
    i = spend0.tolist().index(spend)
    ub_pct = df_bounds.loc[df_bounds['Media'] == media, 'UB'].values[0]
    array_ub.append(min(spend * ub_pct, array_300[len(array_ub)]))
    if spend * ub_pct > array_300[len(array_ub) - 1]:
        ratio = np.round(100 * array_300[len(array_ub) - 1] / (spend * ub_pct), 2)
        print(f"Warning: Entry {i} ({media}) spend (${int(spend * ub_pct)}) exceeds the model 300% spend of {array_300[len(array_ub) - 1]}")
        print(f"{ratio}% of original upper bound")
        print("")
# array_ub = np.array(array_ub)

17.19% of original upper bound

17.19% of original upper bound

59.53% of original upper bound

55.99% of original upper bound

84.47% of original upper bound

47.64% of original upper bound

73.06% of original upper bound

51.15% of original upper bound



In [233]:
len(array_ub)

24

In [194]:
# Step 8: Initialize the iteration trackers
# ================================================================================================================
current_budget = np.sum(array_lb)
remaining_budget = np.sum(spend0) - np.sum(array_lb)
print(f"Benchmark Budget: {np.sum(spend0)}")
print(f"Current Budget: {current_budget}") 
print(f"Remaining Budget: {remaining_budget}")

# Start with the scenario where each media is at their lowest boundary
spend1 = array_lb.copy()

# Create allocation_rank dataframe
allocation_rank = pd.DataFrame({
    'entry': range(len(spend1)),
    'current_CPT': np.zeros(len(spend1)),
    'updates' : np.zeros(len(spend1))
})

for i, (spend, media) in enumerate(zip(spend1, array_media_entries)):
    # Find the closest spend value in df_params_monthly
    closest_row = df_params_monthly[f'S_{i}'].sub(spend).abs().idxmin()
    # Get the corresponding reward and CPT values
    cpt = df_params_monthly.loc[closest_row, f'CPT_{i}']
    # Update current_reward and allocation_rank
    allocation_rank.loc[i, 'current_CPT'] = cpt


# Sort allocation_rank
allocation_rank = allocation_rank.sort_values('current_CPT').reset_index(drop=True)
entries_at_upper_bound = list(allocation_rank.loc[allocation_rank.current_CPT == 0, 'entry'].values)
allocation_rank = allocation_rank[allocation_rank['current_CPT'] > 0]
allocation_rank.current_CPT = allocation_rank.current_CPT.round(0)

Benchmark Budget: 7833195.86598
Current Budget: 1566639.173196
Remaining Budget: 6266556.692784


In [195]:
allocation_rank

,entry,current_CPT,updates
7,4,1.0,0.0
8,12,1.0,0.0
9,2,3.0,0.0
10,10,3.0,0.0
11,7,3.0,0.0
12,6,4.0,0.0
13,14,5.0,0.0
14,15,5.0,0.0
15,20,14.0,0.0
16,1,17.0,0.0


### 2) Allocation starts

In [196]:
# Function for one iterative step
# ================================================================================================================
def update_entry(entry, target_cpt, spend1, allocation_rank, remaining_budget):
    i = allocation_rank.loc[entry, 'entry']
    # Skip if this entry has already hit upper bound
    if i in entries_at_upper_bound:
        print(f"Skipping Entry {i} ({array_media_entries[i]}) as it has already reached upper bound")
        return spend1, allocation_rank, remaining_budget  # Return all three values

    print("Allocating to entry", i, "Media", array_media_entries[i])
    current_spend = spend1[i]
    old_cpt = allocation_rank.loc[entry, 'current_CPT']
    
    # Find the row with target CPT
    target_row = df_params_monthly[f'CPT_{i}'].sub(target_cpt).abs().idxmin()
    next_row = np.minimum(target_row + 1, df_params_monthly.shape[0] - 1)
    
    new_spend = df_params_monthly.loc[next_row, f'S_{i}']

    # Check if new_spend exceeds the upper bound
    if new_spend >= array_ub[i]:
        print(f"Cannot allocate to ${new_spend}, adjusting to upper bound ${array_ub[i]}") 
        new_spend = array_ub[i]
        closest_row = df_params_monthly[f'S_{i}'].sub(new_spend).abs().idxmin()
        new_cpt = df_params_monthly.loc[closest_row, f'CPT_{i}']
        entries_at_upper_bound.append(i)
        print(f"Entry {i} ({array_media_entries[i]}) has reached its upper bound and will not be allocated any more budget.")
    else:
        new_cpt = df_params_monthly.loc[next_row, f'CPT_{i}']

    spend_increase = new_spend - current_spend
    
    if spend_increase > remaining_budget:
        # Insufficient remaining budget
        new_spend = current_spend + remaining_budget
        closest_row = df_params_monthly[f'S_{i}'].sub(new_spend).abs().idxmin()
        new_cpt = df_params_monthly.loc[closest_row, f'CPT_{i}']
        allocated_budget = remaining_budget
        remaining_budget = 0
    else:
        allocated_budget = spend_increase
        remaining_budget -= spend_increase

    spend1[i] = new_spend
    if spend1[i] >= array_ub[i] - 0.01:  # Adding small tolerance for floating point comparison
        if i not in entries_at_upper_bound:
            entries_at_upper_bound.append(i)
            print(f"Entry {i} ({array_media_entries[i]}) has reached its upper bound and will not be allocated any more budget.")

    # FIXED: Always update the CPT to the actual value based on the new spend
    allocation_rank.loc[entry, 'current_CPT'] = new_cpt
    allocation_rank.loc[entry, 'updates'] += 1

    # Find the media with the target CPT for logging purposes
    try:
        target_entry = allocation_rank[allocation_rank['current_CPT'] == target_cpt].iloc[0].name
        target_media_index = allocation_rank.loc[target_entry, 'entry']
        target_media = array_media_entries[target_media_index]
    except:
        target_media = "target"  # If no exact match found

    print(f"Media: {array_media_entries[i]} (Entry {i})" )
    print(f"Target CPT: {target_cpt:.4f} (from {target_media})")
    print(f"Spend: {current_spend:.2f} >>> {new_spend:.2f}  (allocated {spend_increase:.2f})")
    print(f"CPT: {old_cpt:.2f} >>> {new_cpt:.2f}")
    print(f"Remaining budget: {remaining_budget:.2f}")
    print("--------------------")

    return spend1, allocation_rank, remaining_budget

In [197]:
iteration = 1

In [198]:
while remaining_budget > 0:
    print(f"\nIteration {iteration}:")

    # Add safety checks
    if iteration > 2000:  # Prevent infinite loops
        print("Maximum iterations reached. Exiting optimization loop.")
        break
    
    if remaining_budget < 100:  # Stop if remaining budget is very small
        print(f"Remaining budget (${remaining_budget:.2f}) is too small. Ending allocation.")
        break

    # Round CPTs to improve numerical stability
    allocation_rank['current_CPT'] = allocation_rank['current_CPT'].round(0)

    # Re-sort allocation_rank by CPT every iteration
    allocation_rank = allocation_rank.sort_values('current_CPT')

    # Create a filtered view for allocation decisions, excluding entries at upper bounds
    valid_entries = ~allocation_rank.entry.isin(entries_at_upper_bound)
    if not any(valid_entries):
        print("All entries have reached their upper bounds. Ending allocation process.")
        break
    remaining_entries = allocation_rank[valid_entries].copy()

    # Verify we have entries to work with
    if remaining_entries.empty:
        print("No more entries available for allocation. Ending process.")
        break

    # Find lowest CPT and entries with that CPT
    lowest_cpt = remaining_entries['current_CPT'].min() if not remaining_entries.empty else 0
    lowest_cpt_entries = remaining_entries[remaining_entries['current_CPT'] == lowest_cpt]

    # Log current state for debugging
    # print(f"Current allocation state:")
    # print(allocation_rank)
    # print(f"remaining entries: {remaining_entries}")
    if iteration > 300:
        print(f"Current allocation state:")
        print(allocation_rank)
    print(f"Lowest CPT: {lowest_cpt}")
    print(f"Entries with lowest CPT: {lowest_cpt_entries.entry.tolist()}")            

    # Deciding running case:
    special_case = 0
    if len(remaining_entries) == 0:
        special_case = 1
    if remaining_entries['current_CPT'].nunique() == 1 and len(remaining_entries) > 1:
        special_case = 2
    if len(remaining_entries) == 1:
        special_case = 3
    if len(lowest_cpt_entries) > 1 and remaining_entries['current_CPT'].nunique() > 1:
        special_case = 4


    # Special Case 1: All entries have reached their upper bounds
    # .............................................................................................................
    if special_case == 1:
        print("Running Special Case 1")
        print("All entries have reached their upper bounds. Ending allocation process.")
        break

    # Special Case 2: All entries have the same CPT
    # .............................................................................................................
    if special_case == 2:
        print("Running Special Case 2")
        print("All remaining entries have the same CPT. Allocating fixed amount to entry with most room.")

        # Calculate remaining capacity for each entry
        remaining_capacity = []
        for idx in remaining_entries.index:
            i = allocation_rank.loc[idx, 'entry']
            capacity = array_ub[i] - spend1[i]
            remaining_capacity.append({
                'entry_idx': idx,
                'entry': i,
                'capacity': capacity
            })

        # Sort by remaining capacity (descending)
        remaining_capacity = sorted(remaining_capacity, key=lambda x: x['capacity'], reverse=True)

        # Select entry with most capacity
        selected = remaining_capacity[0]
        entry = selected['entry_idx']
        i = selected['entry'] 
        print(f"Selected {array_media_entries[i]} (Entry {i}) with remaining capacity of ${selected['capacity']:.2f}")

        # Cap allocation at 3000 or distance to upper bound
        distance_to_upper = array_ub[i] - spend1[i]
        max_allocation = min(3000, distance_to_upper, remaining_budget)
        print(f"Allocating {max_allocation:.2f} to {array_media_entries[i]} (Entry {i})")

        # Update spend and lookup new reward/CPT
        current_spend = spend1[i]
        spend1[i] += max_allocation
        remaining_budget -= max_allocation

        # Find closest row in df_params_monthly for the new spend level
        closest_row = df_params_monthly[f'S_{i}'].sub(spend1[i]).abs().idxmin()
        new_cpt = df_params_monthly.loc[closest_row, f'CPT_{i}']

        # Update allocation_rank
        allocation_rank.loc[entry, 'current_CPT'] = new_cpt
        allocation_rank.loc[entry, 'updates'] += 1

        if spend1[i] >= array_ub[i] - 0.01:  # Adding small tolerance for floating point comparison
            entries_at_upper_bound.append(i)
            print(f"Entry {i} ({array_media_entries[i]}) has reached its upper bound and will not be allocated any more budget.")

    # Special case 3: Single entry remaining
    # ...........................................................................................................
    elif special_case == 3:
        print("Running special case 3")
        entry = remaining_entries.index[0]
        i = remaining_entries.loc[entry, 'entry']
        current_spend = spend1[i]
        distance_to_upper = array_ub[i] - current_spend

        if distance_to_upper <= 0.01:  # Adding small tolerance
            entries_at_upper_bound.append(i)
            print(f"Entry {i} ({array_media_entries[i]}) is already at upper bound. Skipping.")
            continue


        # Check if we should be allocating to this entry at all
        # Find if there are any other entries not at upper bound
        other_entries = [e for e in allocation_rank.entry.unique() if e not in entries_at_upper_bound and e != i]
        
        if other_entries and allocation_rank.loc[entry, 'current_CPT'] > allocation_rank[allocation_rank.entry.isin(other_entries)]['current_CPT'].min():
            print(f"Found better entries with lower CPT. Skipping allocation to entry {i}.")
            # Re-sort and continue to next iteration
            continue

        # Allocate in smaller chunks to allow for CPT re-evaluation
        max_single_allocation = min(remaining_budget / 4, distance_to_upper / 4)
        if max_single_allocation < 250:  # If allocation becomes too small
            max_single_allocation = min(remaining_budget, distance_to_upper)

        spend1[i] += max_single_allocation
        remaining_budget -= max_single_allocation

        # Update CPT after allocation
        closest_row = df_params_monthly[f'S_{i}'].sub(spend1[i]).abs().idxmin()
        new_cpt = df_params_monthly.loc[closest_row, f'CPT_{i}']
        allocation_rank.loc[entry, 'current_CPT'] = new_cpt
        allocation_rank.loc[entry, 'updates'] += 1

        print(f"Media: {array_media_entries[i]} (Entry {i})")
        print(f"Spend: {current_spend:.2f} >>> {spend1[i]:.2f}  (allocated {max_single_allocation:.2f})")
        print(f"CPT: {allocation_rank.loc[entry, 'current_CPT']:.2f}")
        print(f"Remaining budget: {remaining_budget:.2f}")
        
        if spend1[i] >= array_ub[i] - 0.01:  # Adding small tolerance
            entries_at_upper_bound.append(i)
            print(f"Entry {i} ({array_media_entries[i]}) has reached its upper bound and will not be allocated any more budget.")


    # Special case 4: Multiple entries with the same lowest CPT
    # ...........................................................................................................
    # Check for multiple entries with the same lowest CPT
    if special_case == 4:
        print("Running Special Case 4")
        next_cpt = allocation_rank[allocation_rank['current_CPT'] > lowest_cpt]['current_CPT'].min()
        print(f"Multiple entries with lowest CPT {lowest_cpt:.4f}. Updating to next CPT {next_cpt:.4f}")
        
        # Calculate required spend for each entry
        spend_requirements = []
        for entry in lowest_cpt_entries.index:
            i = allocation_rank.loc[entry, 'entry']
            current_spend = spend1[i]
            
            # Find required spend for target CPT
            target_row = df_params_monthly[f'CPT_{i}'].sub(next_cpt).abs().idxmin()
            next_row = np.minimum(target_row + 1, df_params_monthly.shape[0] - 1)
            required_spend = df_params_monthly.loc[next_row, f'S_{i}'] - current_spend
            
            # Check if required spend exceeds upper bound
            max_possible_spend = array_ub[i] - current_spend
            if required_spend > max_possible_spend:
                required_spend = max_possible_spend

            spend_requirements.append({
                'entry': entry,
                'required_spend': required_spend,
                'media_index': i
            })

        # Sort by required spend
        spend_requirements = sorted(spend_requirements, key=lambda x: x['required_spend'])
        print(spend_requirements)
        
        # Process entries in order of required spend
        for req in spend_requirements:
            # Skip if the entry is already at upper bound
            if req['media_index'] in entries_at_upper_bound:
                continue
            
            # Check if we have enough budget for this entry
            if req['required_spend'] <= 0 or req['required_spend'] > remaining_budget:
                # If not enough budget, just allocate what's left
                if remaining_budget > 0:
                    i = req['media_index']
                    entry = req['entry'] 
                    current_spend = spend1[i] 
                    spend1[i] += remaining_budget  
                
                    # Find closest CPT for new spend
                    closest_row = df_params_monthly[f'S_{i}'].sub(spend1[i]).abs().idxmin()
                    new_cpt = df_params_monthly.loc[closest_row, f'CPT_{i}'] 

                    # Update allocation_rank 
                    allocation_rank.loc[entry, 'current_CPT'] = new_cpt 
                    allocation_rank.loc[entry, 'updates'] += 1  

                    print(f"Media: {array_media_entries[i]} (Entry {i})")
                    print(f"Insufficient budget for target CPT. Allocating remaining ${remaining_budget:.2f}")
                    print(f"Spend: {current_spend:.2f} >>> {spend1[i]:.2f}")
                    print(f"CPT: {allocation_rank.loc[entry, 'current_CPT']:.2f}")

                    if spend1[i] >= array_ub[i] - 0.01: 
                        entries_at_upper_bound.append(i)
                        print(f"Entry {i} ({array_media_entries[i]}) has reached its upper bound and will not be allocated any more budget.")
                    
                    remaining_budget = 0
                break

            results = update_entry(req['entry'], next_cpt, spend1, allocation_rank, remaining_budget)
            spend1, allocation_rank, remaining_budget = results 

            if remaining_budget <= 0:
                print("Stopping allocation as goal has been reached or budget depleted.")
                break





            results = update_entry(req['entry'], next_cpt, spend1, allocation_rank, remaining_budget)
            spend1 = results[0]
            allocation_rank = results[1]
            remaining_budget = results[2]
            if remaining_budget <= 0: 
                print("Stopping allocation as goal has been reached or budget depleted.")
                break


    # Normal Case: Update the entry with lowest CPT
    # ...........................................................................................................
    if special_case == 0:
        # Normal case: update the entry with the lowest CPT
        print("Running Normal Case")
        entry = remaining_entries.index[0]
        # Look up next CPT from full allocation_rank, not just remaining_entries
        entry_cpt = allocation_rank.loc[entry, 'current_CPT']


        # FIXED: Make sure there is at least one higher CPT to target
        higher_cpts = allocation_rank[allocation_rank['current_CPT'] > entry_cpt]['current_CPT']
        if higher_cpts.empty:
            # If no higher CPT exists, create a reasonable target
            i = allocation_rank.loc[entry, 'entry']
            current_spend = spend1[i]
            # Find next higher CPT in df_params_monthly
            current_row = df_params_monthly[f'CPT_{i}'].sub(entry_cpt).abs().idxmin()
            if current_row < df_params_monthly.shape[0] - 1:
                next_row = current_row + 10
                next_cpt = df_params_monthly.loc[next_row, f'CPT_{i}']
            else:
                # Add 20% to the current CPT as a fallback 
                next_cpt = entry_cpt * 1.2 
        else: 
            next_cpt = higher_cpts.min() 
        
        results = update_entry(entry, next_cpt, spend1, allocation_rank, remaining_budget)
        spend1, allocation_rank, remaining_budget = results

    # Recheck all entries to see if any have reached their upper bounds
    for idx in allocation_rank.index:
        i = allocation_rank.loc[idx, 'entry']
        if i not in entries_at_upper_bound and spend1[i] >= array_ub[i] - 0.01:
            entries_at_upper_bound.append(i)
            print(f"Entry {i} ({array_media_entries[i]}) has reached its upper bound and will not be allocated any more budget.")
    
    iteration += 1





Iteration 1:
Lowest CPT: 1.0
Entries with lowest CPT: [4, 12]
Running Special Case 4
Multiple entries with lowest CPT 1.0000. Updating to next CPT 3.0000
[{'entry': 7, 'required_spend': 800.4, 'media_index': 4}, {'entry': 8, 'required_spend': 1111.69475, 'media_index': 12}]
Allocating to entry 4 Media AFF
Cannot allocate to $1710.12475, adjusting to upper bound $870.0
Entry 4 (AFF) has reached its upper bound and will not be allocated any more budget.
Media: AFF (Entry 4)
Target CPT: 3.0000 (from BAN)
Spend: 69.60 >>> 870.00  (allocated 800.40)
CPT: 1.00 >>> 2.28
Remaining budget: 6265756.29
--------------------
Skipping Entry 4 (AFF) as it has already reached upper bound
Allocating to entry 12 Media AFF
Media: AFF (Entry 12)
Target CPT: 3.0000 (from BAN)
Spend: 244.00 >>> 1355.69  (allocated 1111.69)
CPT: 1.00 >>> 3.01
Remaining budget: 6264644.60
--------------------
Allocating to entry 12 Media AFF
Media: AFF (Entry 12)
Target CPT: 3.0000 (from BAN)
Spend: 1355.69 >>> 1355.69  (all

In [201]:
df_bounds

,Media,LB,UB
0,NTV,0.2,2.5
1,STR,0.2,2.5
2,BAN,0.2,2.5
3,RAD,0.2,2.5
4,AFF,0.2,2.5
5,CEM,0.2,2.5
6,SEM,0.2,2.5
7,Total_Social,0.2,2.5


# Minimizer

In [ ]:
# Inputs

spend_plan = scenario1
planning_months = [7, 8, 9]
planning_year = 2025 
target_year = 2026
target_months = [1, 2, 3]
reward_goal = 

# Scenrio Plots

In [53]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

In [54]:
l1 = [1, 2, 3]
l2 = [2, 1, 2]
[max(l1[i], l2[i]) for i in np.arange(len(l1))]

[2, 2, 3]

In [55]:
np.arange(3)

array([0, 1, 2])

You can find the intersection of two sets using the `intersection()` method or the `&` operator in Python. Here is an example:

```python
# Define two sets
set1 = {1, 2, 3, 4, 5}
set2 = {4, 5, 6, 7, 8}

# Find the intersection using the intersection() method
intersection_set = set1.intersection(set2)
print(intersection_set)

# Find the intersection using the & operator
intersection_set = set1 & set2
print(intersection_set)
```

Both methods will give you the intersection of the two sets.

In [71]:
def scenario_plots(scenarios, metrics, channels, colors, title, ylabel1, ylabel2, currency_symbol):
    # **********************************************************************************
    # Read numbers
    # **********************************************************************************
    board1 = scenarios[0]
    board2 = scenarios[1]

    metric1 = metrics[0]
    metric2 = metrics[1]

    # Drop medias with no spending
    # ******************************************************************************
    drops = []
    set1 = set(board1.loc[board1[metric1] == 0, 'Media'].values) 
    set2 = set(board2.loc[board2[metric1] == 0, 'Media'].values)
    drops.append(list(set1.intersection(set2)))
    board1 = board1[~board1['Media'].isin(drops[0])]
    board2 = board2[~board2['Media'].isin(drops[0])] 
    channels = [x for x in channels if x not in drops[0]]


    # Set up metrics
    # ********************************************************************************** 
    metric1_s1 = board1[metric1].values
    metric1_s2 = board2[metric1].values
    max_metric1 =  max(max(metric1_s1), max(metric1_s2))

    metric2_s1 = board1[metric2].values
    metric2_s2 = board2[metric2].values
    max_metric2 = max(max(metric2_s1), max(metric2_s2))

    metric1_compare = [(s2 - s1) / s1 * 100 for s1, s2 in zip(metric1_s1, metric1_s2)]
    metric2_compare = [(s2 - s1) / s1 * 100 for s1, s2 in zip(metric2_s1, metric2_s2)]

    color1_s1 = colors[0]
    color1_s2 = colors[1]
    color2_s1 = colors[2]
    color2_s2 = colors[3]

    default_textpos = 0.5 * max_metric1

    # Create figure with secondary y-axis
    fig = make_subplots(specs=[[{"secondary_y": True}]])

    # **********************************************************************************
    # The bar plots
    # **********************************************************************************
    # Scenario 1
    # ....................................................
    fig.add_trace(
        go.Bar(
            x= channels,
            y= metric1_s1,
            name= "1",
            marker_color= color1_s1
        ),
        secondary_y=False,
    )

    # Scenario 2
    # ....................................................
    fig.add_trace(
        go.Bar(
            x= channels,
            y= metric1_s2,
            name= "2",
            marker_color= color1_s2
        ),
        secondary_y=False,
    )




    # **********************************************************************************
    # Line plots
    # **********************************************************************************
    # Scenario 1
    # ....................................................
    fig.add_trace(
        go.Scatter(
            x=channels,
            y= metric2_s1,
            name="MROAS 1",
            line=dict(
                color= color2_s1, 
                width=3, dash='dash',
                shape='spline',  # This creates a smooth curve
                smoothing=1.3    # Adjust smoothing intensity (0.5-1.5 range works well)
                ),
            marker=dict(
                size=10,         # Larger marker size
                color=color2_s1,
                line=dict(
                    width=1,
                    color='white'
                )
            )

        ),
        secondary_y=True,
    )

    # Scenario 2
    # ....................................................
    fig.add_trace(
        go.Scatter(
            x=channels,
            y= metric2_s2,
            name="MROAS 2",
            line=dict(color= color2_s2, 
                      width=3, dash='dash',
                      shape='spline',  # This creates a smooth curve
                      smoothing=1.3    # Adjust smoothing intensity (0.5-1.5 range works well)
                ),
            marker=dict(
                size=10,         # Larger marker size
                color=color2_s2,
                line=dict(
                    width=1,
                    color='white'
                )
            )
        ),
        secondary_y=True,
    )




    # **********************************************************************************
    # Costmetics
    # **********************************************************************************

    # Annotations for metric 1 change
    for i, channel in enumerate(channels):
        change = metric1_compare[i]
        
        # Determine color based on change
        color = "green" if change >= 0 else "red"
        
        # Format text with plus/minus sign and percentage
        if change >= 0:
            text = f"+{change:.1f}%"  # Add plus sign for positive changes
        else:
            text = f"{change:.1f}%"   # Negative sign is automatically included
        
        # Improved positioning logic
        current_value = max(metric1_s1[i], metric1_s2[i])
        
        # If the value is very small (less than 5% of the maximum), use a fixed position
        if current_value < 0.03 * max_metric1:
            ypos = 0.12 * max_metric1  # Position at 15% of max height for very small values
        # If it's the maximum value, add a bit more space
        elif current_value >= 0.95 * max_metric1:
            ypos = 1.05 * max_metric1  # Position at 110% of max for the largest values
        # For medium values, position proportionally
        else:
            ypos = current_value + (0.125 * max_metric1)  # Position above the bar with consistent spacing
        
        # Add the annotation without arrows
        fig.add_annotation(
            x=channel,
            y=ypos, 
            text=text,
            showarrow=False,  # No arrow
            font=dict(
                color=color, 
                size=14,      # Slightly larger font for better visibility
                weight='bold' # Make it bold for emphasis
            ),
            align='center',
            bgcolor='rgba(255,255,255,0.7)',  # Semi-transparent white background
            bordercolor=color,
            borderwidth=1,
            borderpad=3
        )




    fig.update_layout(
        # Wider plot for spacing
        width=1300,
        height=700,
        # Extra large left margin
        margin=dict(t=80, r=50, b=100, l=150),
        # Title styling
        title=dict(
            text= title,
            font=dict(
                size=28,
                color= color1_s2,
                weight='bold'
            ),
            x=0.5
        ),
        # Other layout
        barmode='group',
        legend=dict(
            orientation="h",
            yanchor="bottom", 
            y=-0.15,
            xanchor="center",
            x=0.5
        ),
        paper_bgcolor='white',
        plot_bgcolor='white'
    )

    # Set x-axis properties
    fig.update_xaxes(
        title_text="",
        showgrid=False,
        showline=True,
        linewidth=2,
        linecolor='lightgray'
    )

    # Set y-axes properties
    fig.update_yaxes(
        title_text= ylabel1,
        title_font=dict(size=16),
        range=[0, 1.2 * max(max(metric1_s2), max(metric1_s1))],
        showgrid=True,
        gridcolor='lightgray',
        secondary_y=False,
        tickformat=','
    )

    fig.update_yaxes(
        title_text= ylabel2,
        title_font=dict(size=16),
        range=[0, 1.2 * max(max(metric2_s2), max(metric2_s1))],
        showgrid=False,
        secondary_y=True,
        tickprefix= currency_symbol,
        ticksuffix='.0'
    )

    # Show the figure
    fig.show()

In [73]:
build_plan_summary(scenario2, 2024, 0.9, ticket_price).iloc[1:, :]

,Media,Total Spend,Total Attendance,Cost per Attendance,Marginal Cost per Attendance,ROAS,MROAS
1,NTV,5678942,331380,17.1,62.7,6.1,1.7
2,STR,2205421,149220,14.8,34.3,7.1,3.1
3,BAN,410196,52691,7.8,16.3,13.5,6.4
4,RAD,265052,16515,16.0,29.7,6.5,3.5
5,AFF,77974,12020,6.5,11.0,16.2,9.6
6,CEM,0,0,NaN,NaN,NaN,NaN
7,SEM,1075893,118347,9.1,19.7,11.5,5.3
8,Total_Social,3902440,346163,11.3,26.9,9.3,3.9


In [72]:
scenario_plots(
    scenarios = [build_plan_summary(scenario2, 2024, 0.9, ticket_price).iloc[1:, :], 
                 build_plan_summary(scenario1, 2024, 0.9, ticket_price).iloc[1:, :]],

    metrics = ['Total Spend', 'MROAS'],

    channels = media_list,

    colors = ['rgb(174, 139, 113)', 
            'rgb(140, 63, 12)',
            'rgb(174, 139, 113)',
            'rgb(140, 63, 12)'
            
            ],

    title = "Media budget & MROAS variation per touchpoint", 

    ylabel1 = "", ylabel2= "", currency_symbol = "£" 
)[0]

TypeError: 'NoneType' object is not subscriptable

In [25]:
scenario_plots(
    scenarios = [build_plan_summary(scenario1, 2024, 0.9, ticket_price).iloc[1:, :], 
                 build_plan_summary(scenario2, 2024, 0.9, ticket_price).iloc[1:, :]],

    metrics = ['Total Attendance', 'Cost per Attendance'],

    channels = media_list,

    colors = ['rgb(188, 214, 150)', 
            'rgb(36, 84, 40)',
            'rgb(188, 214, 150)', 
            'rgb(36, 84, 40)'
            ],

    title = "Incremental attendance & CPA evolution", 

    ylabel1 = "", ylabel2= "", currency_symbol = "£" 
)

NameError: name 'scenario_plots' is not defined

# Labeled plot

In [4]:
import plotly.graph_objects as go

# Create a figure
fig = go.Figure()

# Add the gauge (just the arc/dial part)
fig.add_trace(go.Indicator(
    mode="gauge",
    value=11,
    gauge={
        'axis': {
            'range': [0, 100],
            'visible': True,
            'showticklabels': False,
            'tickwidth': 1,
            'tickcolor': "white"
        },
        'bar': {'color': "white"},
        'bgcolor': "rgba(0,0,0,0)",
        'borderwidth': 0,
        'bordercolor': "rgba(0,0,0,0)",
        'steps': [],
        'threshold': {
            'line': {'color': "white", 'width': 4},
            'thickness': 0.75,
            'value': 11
        }
    },
    domain={'x': [0, 1], 'y': [0, 0.4]}  # Position the gauge at the bottom
))

# Add the text elements with proper spacing
fig.add_annotation(
    text="<b>CPA variation</b>",
    x=0.5, y=0.8,  # Position at the top
    showarrow=False,
    font=dict(size=16, color="white")
)

fig.add_annotation(
    text="€1.4",
    x=0.5, y=0.65,  # Position below the title with space
    showarrow=False,
    font=dict(size=24, color="white")
)

fig.add_annotation(
    text="<b style='color:red'>11%</b>",
    x=0.5, y=0.5,  # Position below the value with space
    showarrow=False,
    font=dict(size=28, color="red")
)

# Update layout with dark background
fig.update_layout(
    paper_bgcolor="#333333",  # Dark gray background
    plot_bgcolor="rgba(0,0,0,0)",
    height=400,
    width=400,
    margin=dict(t=120, b=80, l=40, r=40),  # Add more margin to ensure everything fits
    showlegend=False
)

# Show the figure
fig.show()

# To save:
# fig.write_image("gauge_chart.png")
# fig.write_html("gauge_chart.html")

In [3]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Create figure with customized gauge
fig = go.Figure()

# Add the gauge
fig.add_trace(go.Indicator(
    mode="gauge",
    value=11,  # The percentage value (11%)
    gauge={
        'axis': {
            'range': [None, 100], 
            'visible': True,
            'showticklabels': False,
            'tickwidth': 1, 
            'tickcolor': "white"
        },
        'bar': {'color': "white"},
        'bgcolor': "rgba(0,0,0,0)",
        'borderwidth': 0,
        'bordercolor': "rgba(0,0,0,0)",
        'steps': [],
        'threshold': {
            'line': {'color': "white", 'width': 4},
            'thickness': 0.75,
            'value': 11
        }
    },
    domain={'x': [0, 1], 'y': [0, 0.6]}
))

# Add the textual components separately for more control
fig.add_annotation(
    text="<b>CPA variation</b>",
    x=0.5, y=0.7,
    showarrow=False,
    font=dict(size=16, color="white")
)

fig.add_annotation(
    text="€1.4",
    x=0.5, y=0.6,
    showarrow=False,
    font=dict(size=24, color="white")
)

fig.add_annotation(
    text="11%",
    x=0.5, y=0.4,
    showarrow=False,
    font=dict(size=24, color="red")
)

# Update layout
fig.update_layout(
    paper_bgcolor="rgba(0,0,0,0)",
    plot_bgcolor="rgba(0,0,0,0)",
    margin=dict(t=100, b=0, l=0, r=0),
    height=300,
    width=300,
    showlegend=False
)

# Show the figure
fig.show()

# To save as an image or HTML file:
# fig.write_image("gauge_chart.png")
# fig.write_html("gauge_chart.html")